In [94]:
import spacy
import urllib.request
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request
import re
from nltk.corpus import stopwords
# Load only once
try:
    if(nlp):
        pass
except:
    nlp = spacy.load("en_core_web_md")

# Load only once
try:
    if(sr[0]):
        pass
except:
    sr = stopwords.words('english')


# Only the visible text is to be retrieved
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    title = soup.title.string
    return u" ".join(t.strip() for t in visible_texts), title

def tokenize(text):
    t0 = []
    # Split the text into words separated by any of these chars
    p = re.compile('[-_\", {};:=?\[\]\(\)\'.]')
    
    # Get the word list into an array, t
    t = p.split(text)
    lt = len(t)

    # Add the 4+ chars words into an array, t1. Discard the smaller words
    for i in range(lt):
        w = t[i]
        lw = len(w)
        if(lw > 3):
            t0.append(w)
    return t0

def Lemmatise(text):
# Implementing lemmatization
    p = re.compile('[a-zA-Z]')
    lt = []
    lem = nlp(text)
    # finding lemma for each word
    for word in lem:
#        print(word.text, word.lemma_)
        if(p.findall(str(word))):
            lemma = word.lemma_.lower() 
            if(len(lemma) > 3):
                if str(lemma) not in (sr):
                    lt.append(lemma)
#    print(t1)
    return lt
def CountWords(t):
    global counts
    from collections import Counter
    counts = Counter(t)
#    print(counts)

def ReadUrls(i, j):
    url1 = urls[i]
    url2 = urls[j]
#    print(urls[i], urls[j])
    global html1, html2, t1, t2, t1w, t2w, t1t, t2t
    t1 = []
    t2 = []
    response =  urllib.request.urlopen(url1)
    html1 = response.read()
    response =  urllib.request.urlopen(url2)
    html2 = response.read()
    text1, title1 = text_from_html(html1)
    t1 = Lemmatise(text1)
    t1t = Lemmatise(str(title1))
    text2, title2 = text_from_html(html2)
    t2 = Lemmatise(text2)
    t2t = Lemmatise(str(title2))

    # Make a list of words and their counts. We will take the top 10 for further analysis
    CountWords(t1)
    t1w = []
    for key,val in counts.items():
        item = str(val) + ':' + str(key.lower())
        t1w.append(item)
    t1w.sort(key=lambda fname: int(fname.split(':')[0]), reverse=True)
    CountWords(t2)
    t2w = []
    for key,val in counts.items():
        item = str(val) + ':' + str(key.lower())
        t2w.append(item)
    t2w.sort(key=lambda fname: int(fname.split(':')[0]), reverse=True)
    
def GetDocSegments():
    global doc1_segs, doc2_segs, l, m
    # Split and save the text as 100 word segments
    doc1_segs = []
    doc2_segs = []
    text = ''
    # Split t1 and t2 into text of 100 words
    l = len(t1)
    n = int(l/100)
    for i in range(n):
        b = i*100
        e = b+100
        for j in range(b,e):
            text += t1[j] + ' '
        doc1_segs.append(text)
        text = ''

    l = len(t2)
    n = int(l/100)
    for i in range(n):
        b = i*100
        e = b+100
        for j in range(b,e):
            text += t2[j] + ' '
        doc2_segs.append(text)
        text = ''

    l = len(doc1_segs)    
    m = len(doc2_segs) 
#    print(l,m) # The number of 100-word strings in each

def CalculateSimilarity(i,j,nurls):
    sim = 0.00
    sims = []
    tot = 0.00
    k = nurls
    if (l < k):
        k = l
    if (m < k):
        k = m
    if(k == 0):
        return 0
    for i in range(k):
        for j in range(k):
            doc1 = nlp(doc1_segs[i])
            doc2 = nlp(doc2_segs[j])
            sim = doc1.similarity(doc2)
            sims.append(sim)
    sims.sort(reverse=True)
    for i in range(k):
        tot += sims[i]
    avg = tot/k
    return avg
#    print(avg)

def CalculateWordFrequency():
    tw1f = []
    tw2f = []
# ------------t1--------------
    lw = len(t1w)
    tot = 0
    # Count the total number of words
    for i in range (0,3):
        fields = t1w[i].split(":")
        tot += int(fields[0])
    # Take the percentage of top 10 words in t1w
    for i in range (0,3):
        fields = t1w[i].split(":")
        c = int(fields[0])
        w = fields[1]
        f = str(round(float(c/tot*100),2))
        fw = f + ":" + w
        tw1f.append(fw)
# ------------t2--------------
    lw = len(t2w)
    tot = 0
    # Count the total number of words
    for i in range (0,3):
        fields = t2w[i].split(":")
        tot += int(fields[0])
    for i in range (0,3):
        fields = t2w[i].split(":")
        c = int(fields[0])
        w = fields[1]
        f = str(round(float(c/tot*100),2))
        fw = f + ":" + w
        tw2f.append(w)
# Compare the two to get the word frequency
    lw1 = len(tw1f)
    lw2 = len(tw2f)
    mf = 0.00
    for i in range(lw2):
        w2 = tw2f[i]
        tf = 0.00
        for j in range(lw1):
            fields = tw1f[j].split(":")
            f1 = float(fields[0])
            w1 = fields[1]
            tf += f1
            if(w1 == w2):
#                print(w1,w2)
                mf += f1
    freq = round(mf/tf,2)
#    print(mf, tf, freq)
            
#    print(tw1f)
#    print(tw2f)
    return freq
#        wordfreq += fw + " "
#    print(wordfreq)
    
# Globals
urls_0 = [
"https://en.wikipedia.org/wiki/Machine_learning", # 0
"https://www.sas.com/en_au/insights/analytics/machine-learning.html", # 1
"https://en.wikipedia.org/wiki/SpaceX", # 2
"https://en.wikipedia.org/wiki/London", # 3
"https://en.wikipedia.org/wiki/England",# 4
"https://www.webgenie.com/details.html", # 5
"https://en.wikipedia.org/wiki/Titanic_(1997_film)", # 6
"https://en.wikipedia.org/wiki/Titanic_(1953_film)", # 7
"https://simple.wikipedia.org/wiki/Titanic_(1997_movie)", # 8
"https://en.wikipedia.org/wiki/Star_Wars", # 9
"https://en.wikipedia.org/wiki/List_of_Star_Wars_films" # 10
]
urls = [
"https://www.ands.org.au/working-with-data/metadata/geospatial-data-and-metadata",#0
"https://www.ands.org.au/guides/geospatial",#1
"https://www.gislounge.com/difference-gis-geospatial/",#2
"https://gisgeography.com/what-is-geodata-geospatial-data/",#3
"https://www.mathworks.com/help/map/what-is-geospatial-data.html",#4
"https://unimelb.libguides.com/GIS",#5
"https://www.veris.com.au/our-services/geospatial-data-management/"#6
]
nurls = len(urls)


In [95]:
def Title_Match(t1t, t2t):
    tv = 0.00
    for w1 in (t1t):
        for w2 in (t2t):
            if (w1 == w2):
                tv = 0.1
    return tv

In [96]:
def Domain_match(url1, url2):
    dv = 0.00
    f1 = url1.split('/')
    d1 = f1[2].replace('www.','')
    f2 = url2.split('/')
    d2 = f2[2].replace('www.','')
    if(d1 == d2):
        dv = 0.1
    return dv

In [105]:
#-----------------------------------------------------------------------------------------------------------------
# Call the functions
# Read the HTML contents and lemmatise into two arrays, 't1' and 't2'
i = 0
j = 6
ReadUrls(i, j)
#print(t1w[:10])

# Concatenate 100 words each from t1 and t2 into separate strings and add to two arrays, 'doc1_segs' and 'doc2_segs'
GetDocSegments()

# Calculate the cosine similarity
sim = CalculateSimilarity(i,j,nurls)

freq = CalculateWordFrequency()
tv = Title_Match(t1t, t2t) # If any word in titles match, tf=0.1
dv = Domain_match(urls[i], urls[j])
s = round((sim + freq + tv + dv),2)
#s = round((sim + freq),2)
sp = round(s/2.00*100.0,2)
print(t1t, t2t, tv, dv)
print('Score: ', s, 'out of 2.00 (', sp , '%)')

['geospatial', 'datum', 'metadata'] ['geographic', 'information', 'systems', 'system', 'services', 'veris', 'australia'] 0.0 0.0
Score:  0.95 out of 2.00 ( 47.5 %)


In [98]:
h = "#\t"
o = ""
for i in range (nurls):
    h += str(i) + '\t'
#h += '\n'  
print(h)
#print('Blah')
def AddTabs(i):
    global o
    for j in range(i):
        o += '\t'
for i in range (nurls):
    o = str(i)
    AddTabs(i+1)
    for j in range (i,nurls):
        ReadUrls(i, j)
        tv = Title_Match(t1t, t2t) # If any word in titles match, tf=0.1
        dv = Domain_match(urls[i], urls[j])
        GetDocSegments()
        sim = CalculateSimilarity(i,j,nurls)
        freq = CalculateWordFrequency()
        s = round((sim + freq + tv + dv),2)
        o += str(s) + '\t'
#    o += '\n'  
    print(o)


#	0	1	2	3	4	5	6	
0	2.2	1.88	1.05	1.52	1.04	1.53	0.95	
1		2.2	1.32	1.61	1.32	1.63	1.21	
2			2.2	1.27	1.24	1.27	1.69	
3				2.2	1.62	1.83	1.52	
4					2.2	1.35	1.23	
5						2.2	1.55	
6							2.2	
